In [2]:
# added by me to get rid of warning 
import warnings
warnings.filterwarnings("ignore", message=".*Intel MKL WARNING.*")

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import salamander

In [4]:
# load the mutation counts. (mutation type) x (cancer patients)
counts_sbs_appended = pd.read_csv("data/pcawg_breast_sbs_appended.csv", index_col=0)
counts_sbs_appended.shape

(96, 193)

In [5]:
counts_sbs_appended.head()

,SP10084,SP10150,SP10470,SP10563,SP10635,SP10944,SP11045,SP11171,SP11235,SP116331,...,SP9251,SP9433,SP9481,SP96147,SP96163,SP96511,SP9816,SP9930,SP9979,SP41XX
SBS,,,,,,,,,,,,,,,,,,,,,
A[C>A]A,239,61,218,56,35,144,28,153,78,125,...,94,47,45,77,139,268,133,166,32,1065.0
A[C>A]C,199,47,169,33,23,111,31,116,60,62,...,60,36,33,41,106,210,108,159,19,1463.0
A[C>A]G,28,4,26,3,8,19,3,22,10,15,...,10,9,7,10,17,38,17,20,3,620.0
A[C>A]T,222,34,194,22,24,94,21,125,43,47,...,72,34,26,36,115,209,136,120,23,891.0
C[C>A]A,163,44,206,41,24,90,28,110,78,104,...,57,44,31,50,102,230,120,176,15,1070.0


# NMF with KL-divergence loss

In [6]:
# 7 signatures was set
n_signatures = 7

# Fit the NMF model
# NMF with Poisson noise
model = salamander.KLNMF(
    n_signatures=n_signatures,
    max_iterations=10000
)
model.fit(counts_sbs_appended, history=True)

In [8]:
# objective function over iterations
model.plot_history(min_iteration=500)

The fitted signatures and exposures can be accessed via $\texttt{model.signatures}$ and $\texttt{model.exposures}$ respecively:

In [9]:
model.signatures.shape

(96, 7)

In [10]:
model.signatures.head()

,Sig1,Sig2,Sig3,Sig4,Sig5,Sig6,Sig7
A[C>A]A,0.000713,0.007343,0.053967,0.002558,0.000820,0.015141,0.015495
A[C>A]C,0.000797,0.010201,0.037007,0.000897,0.001451,0.009696,0.015075
A[C>A]G,0.000154,0.004289,0.002872,0.000162,0.000065,0.002162,0.003071
A[C>A]T,0.000430,0.006187,0.038163,0.001327,0.002301,0.004110,0.016165
C[C>A]A,0.001120,0.007361,0.051828,0.002137,0.001222,0.007419,0.011041


In [12]:
model.exposures.shape

(7, 193)

In [13]:
model.exposures.head()

,SP10084,SP10150,SP10470,SP10563,SP10635,SP10944,SP11045,SP11171,SP11235,SP116331,...,SP9251,SP9433,SP9481,SP96147,SP96163,SP96511,SP9816,SP9930,SP9979,SP41XX
Sig1,374.927938,2424.660026,341.445493,1688.445608,86.635156,3997.617700,1560.530227,146.849977,189.257570,118.335739,...,273.774015,5.067937e+03,1.012139e+00,97.852961,990.895029,783.162629,402.926760,3757.620924,71.494759,2.063912e+03
Sig2,1053.709702,125.993236,669.343746,72.100873,66.038917,362.416535,60.537478,483.193989,116.829122,246.610260,...,189.508657,1.192093e-07,1.192093e-07,101.339361,519.802921,850.984062,435.304874,535.834686,11.921099,1.439673e+05
Sig3,2623.322730,210.442509,2202.985349,268.288393,135.874399,827.770197,189.971534,1219.728359,837.862534,1347.030504,...,633.776959,1.192093e-07,2.201661e+02,465.254867,881.959694,2118.207034,1119.838721,1405.183401,113.550407,4.196944e-07
Sig4,317.132910,2540.073706,826.785956,293.410095,36.212330,2393.722424,1062.773347,176.844443,223.024118,182.164625,...,371.431817,6.671803e+03,4.062226e+01,96.614307,1755.460402,1639.801132,733.774836,4863.144933,39.497711,1.492106e+03
Sig5,94.210755,51.226028,279.737794,25.527713,13.176904,86.543642,13.326019,90.049840,64.190438,119.319788,...,41.221039,4.284768e+01,2.112778e+01,28.196592,213.014231,392.993259,46.484037,296.848766,17.252666,2.681038e+01


The NMF model also comes with visualization methods for the signatures and exposures:

In [14]:
fig, axes = plt.subplots(7, 1, figsize=(8, 10))

model.plot_signatures(annotate_mutation_types=True, axes=axes)

array([<Axes: title={'center': 'Sig1'}>, <Axes: title={'center': 'Sig2'}>,
       <Axes: title={'center': 'Sig3'}>, <Axes: title={'center': 'Sig4'}>,
       <Axes: title={'center': 'Sig5'}>, <Axes: title={'center': 'Sig6'}>,
       <Axes: title={'center': 'Sig7'}>], dtype=object)

In [15]:
fig, ax = plt.subplots(1, 1, figsize=(35, 5))

# stacked barplot of the exposures
model.plot_exposures(ax=ax)

<Axes: title={'center': 'Sample exposures'}>

We can also plot dimensionality reductions of the exposures and highlight the outlier sample heavily treated with chemotherapy.

In [16]:
# check the names of the samples, what are these samples? Biological samples from where these are derived? 
display(model.sample_names.shape, model.sample_names)

(193,)

array(['SP10084', 'SP10150', 'SP10470', 'SP10563', 'SP10635', 'SP10944',
       'SP11045', 'SP11171', 'SP11235', 'SP116331', 'SP116333',
       'SP116335', 'SP116339', 'SP116341', 'SP116343', 'SP116345',
       'SP116347', 'SP116349', 'SP116351', 'SP116353', 'SP116355',
       'SP116357', 'SP116359', 'SP116361', 'SP116363', 'SP116365',
       'SP116367', 'SP116369', 'SP116371', 'SP116373', 'SP116375',
       'SP116377', 'SP116379', 'SP116946', 'SP116947', 'SP116948',
       'SP116963', 'SP116991', 'SP117011', 'SP117032', 'SP117057',
       'SP117078', 'SP117079', 'SP117105', 'SP117108', 'SP117113',
       'SP117126', 'SP117127', 'SP117136', 'SP117162', 'SP117244',
       'SP117245', 'SP117250', 'SP117257', 'SP117312', 'SP117344',
       'SP117354', 'SP117363', 'SP117369', 'SP117370', 'SP117372',
       'SP117378', 'SP117402', 'SP117409', 'SP117454', 'SP117538',
       'SP117552', 'SP117593', 'SP117598', 'SP117614', 'SP117618',
       'SP117629', 'SP117639', 'SP117676', 'SP117710', 'SP1

In [17]:
labels = [
    sample if sample == "SP41XX" else "other"
    for sample in model.sample_names
]

In [18]:
# PCA, t-SNE and UMAP of the sample exposures
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

model.plot_embeddings(method="pca", ax=axes[0], hue=labels)
model.plot_embeddings(method="tsne", ax=axes[1], hue=labels)
model.plot_embeddings(method="umap", ax=axes[2], hue=labels)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


<Axes: xlabel='UMAP1', ylabel='UMAP2'>

In [19]:
# load the COSMIC catalog of known mutational signatures
catalog = pd.read_csv("data/COSMIC_v3.3.1_SBS_GRCh38.csv", index_col=0)
catalog.head()

,SBS1,SBS2,SBS3,SBS4,SBS5,SBS6,SBS7a,SBS7b,SBS7c,SBS7d,...,SBS86,SBS87,SBS88,SBS89,SBS90,SBS91,SBS92,SBS93,SBS94,SBS95
Type,,,,,,,,,,,,,,,,,,,,,
A[C>A]A,0.000876,5.790059e-07,0.020920,0.042451,0.012052,0.000425,0.000067,0.002344,0.004841,0.000040,...,0.002968,0.008946,1.004268e-18,0.032297,0.002222,0.002934,0.011396,0.011628,0.015677,0.038614
A[C>A]C,0.002220,1.455045e-04,0.016343,0.032990,0.009337,0.000516,0.000177,0.000457,0.001135,0.000754,...,0.003735,0.004490,9.890296e-19,0.017495,0.000704,0.052013,0.009653,0.008011,0.024523,0.017212
A[C>A]G,0.000180,5.361861e-05,0.001808,0.016116,0.001908,0.000053,0.000073,0.000192,0.000388,0.000257,...,0.000398,0.006357,1.031355e-18,0.009971,0.000144,0.000209,0.004851,0.001817,0.001627,0.008632
A[C>A]T,0.001265,9.759122e-05,0.012265,0.029663,0.006636,0.000180,0.000249,0.000714,0.001964,0.004051,...,0.003639,0.004941,1.737757e-03,0.020818,0.001771,0.000130,0.007800,0.008457,0.011141,0.023409
C[C>A]A,0.000305,2.053143e-04,0.022376,0.080269,0.007379,0.001800,0.000451,0.001134,0.000108,0.014348,...,0.006076,0.008436,9.930026e-19,0.024492,0.001183,0.008071,0.018450,0.006456,0.079522,0.058394


In [20]:
display(catalog.shape)
display(counts_sbs_appended.shape)

(96, 79)

(96, 193)


Justify your choice: Consider the following factors:

Cosine Similarity: Suitable for identifying overall patterns and broad similarities.

Manhattan Distance: Emphasizes differences in individual mutation frequencies.

KL Divergence: Sensitive to subtle variations in signature profiles.

Spearman Correlation: Focuses on the rank order of mutations, useful if absolute frequencies are less important.

Explain your rationale for choosing

In [21]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine, cityblock  # For cosine and Manhattan distances
from scipy.stats import spearmanr, entropy  # For Spearman correlation and KL divergence

In [22]:
pcawg = model.signatures #for the given breast cancer sbs signs, catalog for reference cosmic 
pcawg
pcawg.to_csv("pcawg.csv", index = True)

In [23]:
from SigProfilerAssignment import Analyzer as Analyze
from scipy.spatial.distance import cosine
from scipy.stats import entropy


In [24]:
# inbuilt package db 

from SigProfilerAssignment import Analyzer as Analyze
Analyze.cosmic_fit(pcawg, output= "/Users/anirudh/Desktop/acb_challenge/Cosmic3.4_inbuiltDB", input_type="matrix", context_type="96",
                   collapse_to_SBS96=True, cosmic_version=3.4, exome=False,
                   genome_build="GRCh37", signature_database=None,
                   exclude_signature_subgroups=None, export_probabilities=False,
                   export_probabilities_per_mutation=False, make_plots=False,
                   sample_reconstruction_plots=False, verbose=False)

Assigning COSMIC sigs or Signature Database ...... 
|████████████████████████████████████████| 7/7 [100%] in 0.5s (13.55/s) 


 
Your Job Is Successfully Completed! Thank You For Using SigProfilerAssignment.
 


In [36]:
# given ref signatures from cosmic 

from SigProfilerAssignment import Analyzer as Analyze
Analyze.cosmic_fit(pcawg, output= "/Users/anirudh/Desktop/acb_challenge/COSMIC3.3.1_GivenRefSig", input_type="matrix", context_type="96", collapse_to_SBS96=True,
                   exome=False,
                   genome_build="GRCh38", signature_database="/Users/anirudh/Desktop/acb_challenge/COSMIC_v3.3.1_SBS_GRCh38.txt",
                   exclude_signature_subgroups=None, export_probabilities=False,
                   export_probabilities_per_mutation=False, make_plots=False,
                   sample_reconstruction_plots=False, verbose=False)

Assigning COSMIC sigs or Signature Database ...... 
|████████████████████████████████████████| 7/7 [100%] in 0.5s (13.57/s) 


 
Your Job Is Successfully Completed! Thank You For Using SigProfilerAssignment.
 


In [26]:
pcawg_T = pcawg.T
pcawg_T

,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,C[C>A]A,C[C>A]C,C[C>A]G,C[C>A]T,G[C>A]A,G[C>A]C,...,C[T>G]G,C[T>G]T,G[T>G]A,G[T>G]C,G[T>G]G,G[T>G]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T
Sig1,0.000713,0.000797,0.000154,0.000430,0.001120,0.000717,2.500184e-04,0.000816,1.192093e-07,0.000944,...,3.808979e-04,0.000315,1.635983e-04,1.809828e-04,2.184910e-04,0.000144,1.192093e-07,0.000099,0.000324,0.000467
Sig2,0.007343,0.010201,0.004289,0.006187,0.007361,0.007623,3.439877e-03,0.013539,8.401739e-03,0.010976,...,4.096106e-03,0.007061,5.888681e-03,3.541505e-03,4.316354e-03,0.005583,1.087688e-02,0.004347,0.005527,0.008760
Sig3,0.053967,0.037007,0.002872,0.038163,0.051828,0.031055,4.279318e-03,0.044311,4.940078e-02,0.020717,...,2.902208e-03,0.004251,8.419442e-04,1.085029e-05,3.737228e-03,0.001748,1.187136e-04,0.000557,0.004713,0.005027
Sig4,0.002558,0.000897,0.000162,0.001327,0.002137,0.000817,1.192093e-07,0.000832,2.244455e-03,0.000001,...,1.192093e-07,0.000135,1.192093e-07,1.192093e-07,1.192093e-07,0.000063,1.025375e-03,0.000268,0.000112,0.000892
Sig5,0.000820,0.001451,0.000065,0.002301,0.001222,0.000962,3.906046e-04,0.004396,7.216683e-04,0.000837,...,1.053847e-02,0.195642,1.220817e-03,4.240454e-03,2.686582e-03,0.050829,6.735546e-03,0.011268,0.007328,0.120094
Sig6,0.015141,0.009696,0.002162,0.004110,0.007419,0.002511,2.275478e-03,0.005701,1.477300e-02,0.005708,...,2.029058e-03,0.002875,8.110605e-04,1.184911e-03,2.863063e-03,0.002009,3.859784e-03,0.002680,0.002346,0.009000
Sig7,0.015495,0.015075,0.003071,0.016165,0.011041,0.015732,2.374172e-03,0.011365,2.080931e-03,0.009213,...,1.102882e-02,0.002795,4.486945e-03,3.494061e-03,1.097155e-02,0.006159,8.982815e-03,0.005863,0.010607,0.012083


In [27]:
#cosine, 
# iterate through the values of pcawg.T and the cols of cosmic 
similarities = pd. DataFrame()
for i in pcawg_T.index:
    for j in catalog.columns:
        similarities.loc[i, j] = 1 - cosine(pcawg_T.loc[i], catalog[j])  # 1-cosine for similarity

In [28]:
display(similarities)
similarities.to_csv("similarities.csv", index = True)

,SBS1,SBS2,SBS3,SBS4,SBS5,SBS6,SBS7a,SBS7b,SBS7c,SBS7d,...,SBS86,SBS87,SBS88,SBS89,SBS90,SBS91,SBS92,SBS93,SBS94,SBS95
Sig1,0.026613,0.991306,0.144961,0.066051,0.284433,0.081633,0.713330,0.274809,0.065394,0.076898,...,0.134402,0.053008,0.008944,0.159290,0.009476,0.096943,0.232344,0.119251,0.144389,0.095849
Sig2,0.072871,0.034728,0.509440,0.323423,0.432734,0.131644,0.153338,0.371104,0.478714,0.163256,...,0.140156,0.186550,0.302146,0.436805,0.854943,0.061698,0.443409,0.442075,0.432277,0.273920
Sig3,0.044688,0.034818,0.619329,0.837586,0.440023,0.156833,0.092095,0.142634,0.205015,0.104814,...,0.211374,0.136413,0.140004,0.621638,0.090759,0.392446,0.397183,0.287311,0.684189,0.896296
Sig4,0.012701,0.083967,0.292486,0.089720,0.167606,0.013326,0.073112,0.028659,0.051821,0.015699,...,0.533014,0.062143,0.022001,0.311500,0.009737,0.096299,0.228799,0.204720,0.136262,0.112014
Sig5,0.009431,0.016081,0.322232,0.073859,0.296355,0.024259,0.020339,0.035831,0.228751,0.129950,...,0.096766,0.075761,0.290498,0.196769,0.112036,0.008422,0.169196,0.519631,0.170500,0.086488
Sig6,0.793844,0.229803,0.467003,0.285233,0.690416,0.795520,0.349801,0.414262,0.163309,0.233060,...,0.219851,0.679314,0.328770,0.415250,0.121645,0.149601,0.587410,0.424948,0.519730,0.383647
Sig7,0.257307,0.082362,0.918503,0.472174,0.866101,0.371975,0.186385,0.294438,0.318987,0.332801,...,0.510353,0.374706,0.486702,0.764558,0.132621,0.056140,0.736111,0.626247,0.667402,0.426210


In [29]:
closest_cosmic_sigs = pd.DataFrame(similarities.idxmax(axis=1))
closest_cosmic_sigs["Sig#"] = ["Sig1", "Sig2", "Sig3","Sig4","Sig5","Sig6","Sig7",]
closest_cosmic_sigs.columns = ['Matched/ Simialr Sig from COSMIC Ref Sig DB',  closest_cosmic_sigs.columns[1]]

closest_cosmic_sigs = closest_cosmic_sigs.reset_index(drop = True)


closest_cosmic_sigs

,Matched/ Simialr Sig from COSMIC Ref Sig DB,Sig#
0,SBS2,Sig1
1,SBS90,Sig2
2,SBS18,Sig3
3,SBS13,Sig4
4,SBS17b,Sig5
5,SBS6,Sig6
6,SBS3,Sig7


In [30]:
closest_cosmic_sigs.shape

(7, 2)

In [31]:
## Analyzing outlier 

lier = counts_sbs_appended[["SP41XX"]]
lier.T



SBS,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,C[C>A]A,C[C>A]C,C[C>A]G,C[C>A]T,G[C>A]A,G[C>A]C,...,C[T>G]G,C[T>G]T,G[T>G]A,G[T>G]C,G[T>G]G,G[T>G]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T
SP41XX,1065.0,1463.0,620.0,891.0,1070.0,1097.0,498.0,1943.0,1231.0,1584.0,...,595.0,1023.0,851.0,513.0,626.0,809.0,1570.0,629.0,797.0,1269.0


In [32]:
lier_similarities = pd.DataFrame()


for i in lier.T.index:
    for j in catalog.columns:
        lier_similarities.loc[i, j] = 1 - cosine(lier.T.loc[i], catalog[j])  # 1-cosine for similarity

print(lier_similarities)

matched_lier_similarities = pd.DataFrame(lier_similarities.idxmax(axis=1))
matched_lier_similarities



            SBS1      SBS2      SBS3      SBS4      SBS5      SBS6     SBS7a  \
SP41XX  0.074738  0.075171  0.520542  0.326809  0.446659  0.135376  0.182535   

          SBS7b     SBS7c     SBS7d  ...     SBS86     SBS87    SBS88  \
SP41XX  0.38188  0.478648  0.165844  ...  0.159152  0.190147  0.30126   

           SBS89     SBS90     SBS91     SBS92     SBS93     SBS94     SBS95  
SP41XX  0.449006  0.852178  0.067904  0.456738  0.449777  0.439532  0.279792  

[1 rows x 79 columns]


,0
SP41XX,SBS90


In [33]:
lier_similarities #max is SBS90

,SBS1,SBS2,SBS3,SBS4,SBS5,SBS6,SBS7a,SBS7b,SBS7c,SBS7d,...,SBS86,SBS87,SBS88,SBS89,SBS90,SBS91,SBS92,SBS93,SBS94,SBS95
SP41XX,0.074738,0.075171,0.520542,0.326809,0.446659,0.135376,0.182535,0.38188,0.478648,0.165844,...,0.159152,0.190147,0.30126,0.449006,0.852178,0.067904,0.456738,0.449777,0.439532,0.279792
